Loading the utilities

In [58]:
import tensorflow as tf
import re
import pandas as pd
import nltk
import numpy as np
import tensorflow_hub as hub
import tensorflow_text as text #prereqprerequisite for bert preprocess
from sklearn.model_selection import train_test_split

In [22]:
!pip install -q tensorflow-text

     |████████████████████████████████| 3.4MB 11.8MB/s 


In [8]:
#stopwords from nltk to remove them from our dataset
from nltk.corpus import stopwords
stop=stopwords.words('english')

Loading the small bert model from tensorflow hub

In [9]:
tfhub_handle_encoder = 'https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-512_A-8/1'
tfhub_handle_preprocess = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [10]:
path_to_file='drive/MyDrive/Stumble/train.tsv'

Dataset

In [11]:
df = pd.read_csv(path_to_file, sep='\t')

In [12]:
df.columns

Index(['url', 'urlid', 'boilerplate', 'alchemy_category',
       'alchemy_category_score', 'avglinksize', 'commonlinkratio_1',
       'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4',
       'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio',
       'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news',
       'lengthyLinkDomain', 'linkwordscore', 'news_front_page',
       'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url',
       'parametrizedLinkRatio', 'spelling_errors_ratio', 'label'],
      dtype='object')

In [13]:
df.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0,0.090774,0,0.245831,0.003883,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0,0.098707,0,0.203490,0.088652,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0,0.072448,0,0.226402,0.120536,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,0.480725,0.0,0,0.095861,0,0.265656,0.035343,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,0.446143,0.0,0,0.024908,0,0.228887,0.050473,1,1,14,0,12032,162,10,0.098765,0.082569,0


cloning the dataframe

In [14]:
vf=df

Removing columns which are not required for determining the label. Referring to the paper : http://cs229.stanford.edu/proj2013/RoySaraiya_StumbleUponClassification.pdf

In [15]:
vf.drop(columns=['url', 'urlid', 'alchemy_category',
       'alchemy_category_score', 'avglinksize', 'commonlinkratio_1',
       'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4',
       'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio',
       'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news',
       'lengthyLinkDomain', 'linkwordscore', 'news_front_page',
       'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url',
       'parametrizedLinkRatio', 'spelling_errors_ratio'], axis=1,inplace=True)
vf.head()

,boilerplate,label
0,"{""title"":""IBM Sees Holographic Calls Air Breat...",0
1,"{""title"":""The Fully Electronic Futuristic Star...",1
2,"{""title"":""Fruits that Fight the Flu fruits tha...",1
3,"{""title"":""10 Foolproof Tips for Better Sleep ""...",1
4,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",0


Cleaning the boilerplate data

In [16]:
def clean_text(x):
  x=re.sub('{"title":"','',x)
  x=re.sub('","body":"',' ',x)
  x=re.sub('","url":"',' ',x)
  x=re.sub('html"}','',x)
  return x

Preprocessing

In [17]:
pd.options.mode.chained_assignment = None
def pre_process(w):
  for i in range(0,len(w)):
    w['boilerplate'][i]=clean_text(w['boilerplate'][i])
  w['boilerplate'] = w['boilerplate'].str.lower()
  w['boilerplate'] = w['boilerplate'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [18]:
pre_process(vf)

In [19]:
vf.head()

,boilerplate,label
0,ibm sees holographic calls air breathing batte...,0
1,fully electronic futuristic starting gun elimi...,1
2,fruits fight flu fruits fight flu | cold & flu...,1
3,10 foolproof tips better sleep period life lot...,1
4,50 coolest jerseys know existed coolest jersey...,0


Splitting Test and Train Dataset

In [38]:
X_train, X_test, y_train, y_test = train_test_split(vf['boilerplate'], vf['label'], test_size=0.2, random_state=42,shuffle=True)

Loading BERT Preprocess and Encoder model from tensorflow_hub
Ref : https://tfhub.dev/google/collections/bert/1

In [24]:
bert_preprocess_model = hub.KerasLayer(tfhub_handle_preprocess)
bert_model = hub.KerasLayer(tfhub_handle_encoder)

Building the model

In [25]:
def BERTClassifier():
  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)
  net = outputs['pooled_output']
  net = tf.keras.layers.Dropout(0.1)(net)
  net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
  return tf.keras.Model(text_input, net)

In [26]:
classifier_model = BERTClassifier()

In [33]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [39]:
epochs = 10
optimizer = tf.keras.optimizers.Adam()

In [34]:
classifier_model.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics)

Training the model

In [40]:
history = classifier_model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=epochs)

Epoch 1/10
185/185 [==============================] - 698s 4s/step - loss: 0.4828 - binary_accuracy: 0.7591 - val_loss: 0.5871 - val_binary_accuracy: 0.7525
Epoch 2/10
185/185 [==============================] - 697s 4s/step - loss: 0.4105 - binary_accuracy: 0.8315 - val_loss: 0.6949 - val_binary_accuracy: 0.7221
Epoch 3/10
185/185 [==============================] - 704s 4s/step - loss: 0.4843 - binary_accuracy: 0.7789 - val_loss: 0.5982 - val_binary_accuracy: 0.7187
Epoch 4/10
185/185 [==============================] - 707s 4s/step - loss: 0.3947 - binary_accuracy: 0.8342 - val_loss: 0.7356 - val_binary_accuracy: 0.7059
Epoch 5/10
185/185 [==============================] - 703s 4s/step - loss: 0.3429 - binary_accuracy: 0.8622 - val_loss: 0.6492 - val_binary_accuracy: 0.7323
Epoch 6/10
185/185 [==============================] - 709s 4s/step - loss: 0.3064 - binary_accuracy: 0.8818 - val_loss: 0.6717 - val_binary_accuracy: 0.7323
Epoch 7/10
185/185 [==============================] - 724s

In [106]:
score= classifier_model.evaluate(X_test, y_test, verbose = 2)

47/47 - 52s - loss: 0.8153 - binary_accuracy: 0.7343


Loading Test Dataset

In [41]:
test_file = pd.read_csv('drive/MyDrive/Stumble/test.tsv', sep='\t')

In [42]:
test_file.drop(columns=['url', 'alchemy_category',
       'alchemy_category_score', 'avglinksize', 'commonlinkratio_1',
       'commonlinkratio_2', 'commonlinkratio_3', 'commonlinkratio_4',
       'compression_ratio', 'embed_ratio', 'framebased', 'frameTagRatio',
       'hasDomainLink', 'html_ratio', 'image_ratio', 'is_news',
       'lengthyLinkDomain', 'linkwordscore', 'news_front_page',
       'non_markup_alphanum_characters', 'numberOfLinks', 'numwords_in_url',
       'parametrizedLinkRatio', 'spelling_errors_ratio'], axis=1,inplace=True)

Saving our model for future use

In [44]:
classifier_model.save('drive/MyDrive/Stumble/MyModel',include_optimizer=False)

INFO:tensorflow:Assets written to: drive/MyDrive/Stumble/MyModel/assets


INFO:tensorflow:Assets written to: drive/MyDrive/Stumble/MyModel/assets


Evaluating with test dataset

In [45]:
reloaded_model = tf.saved_model.load('drive/MyDrive/Stumble/MyModel')

In [61]:
pre_process(test_file)

In [63]:
results = tf.sigmoid(reloaded_model(tf.constant(test_file['boilerplate'])))

In [59]:
test_file['label']=np.nan

In [105]:
for j in range((len(results)-1)):
  if results[j]>(0.9):
    test_file['label'][j]=int(1)
  else: test_file['label'][j]=int(0)

In [103]:
test_file=test_file.fillna(0)
test_file.label.astype(int)

0       1
1       0
2       0
3       1
4       1
       ..
3166    1
3167    1
3168    1
3169    1
3170    0
Name: label, Length: 3171, dtype: int64

In [84]:
test_file.drop('boilerplate',axis=1,inplace=True)

In [104]:
test_file.head()

,urlid,label
0,5865,1.0
1,782,0.0
2,6962,0.0
3,7640,1.0
4,3589,1.0


Saving the Predicted Dataset into a csv file

In [101]:
test_file.to_csv('drive/MyDrive/Stumble/submit3.csv', index=False)